# Public Key Cryptography II

## 1 Elliptic curve over real numbers $\mathbb{R}$

Given the elliptic curve $E\ :\ y^2 =x^3−3x+9$. If this curve is not singular (check this), compute $a$ and $b$, such that $P_1 = (0,a)$ and $P_2 = (2,b)$ lay on $E$. Compute $P = P_1 +P_2$ and draw $E$ together with $P_1$ , $P_2$ and $P$.

## 2 Elliptic curve over real numbers $\mathbb{R}$

Given the elliptic curve $E\ :\ y^2 = x^3 −3x+5$ and point $P = (2,2.65)\in E$.


__Your Task__: Draw $E$, point $P$ and compute $2P$, $4P$ and $8P$. Solve this problem with minimal computational work!

## 3 Singular elliptic curve

Show that the condition $4a^3 +27b^2 \neq 0 \mod p$ is satisfied for the elliptic curve

$$y^2 \equiv x^3 +2x+7 \mod 17$$

For this elliptic curve, the two coefficients are $a,b = 2, 7$, hence the condition is $4\cdot 2^3 + 27\cdot 7^2 \equiv 12 \mod 17$, which means our ECC is non-singular (has no cusps or intersections)

In [1]:
print((4*2**3 + 27 * 7**2)%17)

12


Example of a singular ECC

In [2]:
import numpy as np
import matplotlib.pyplot as plt

a = -3
b = 2
xlist = np.linspace( - 3, 4, 100)
ylist = np.linspace( - 6.0, 6.0, 100)
X, Y = np.meshgrid(xlist, ylist)
Z = Y ** 2 - X ** 3 - a * X - b
plt.contour(X, Y, Z, [0], colors='red') 
plt.grid()
plt.axis('scaled')
plt.show()

<Figure size 640x480 with 1 Axes>

## 4 Elliptic curve over finite field $\mathbb{Z}_7$

Let $E\ :\ y^2 \equiv x^3 +3x+2 \mod 7$ an elliptic curve over $\mathbb{Z}_7$.


__Your Task__:
1. Compute all points on $E$ over $\mathbb{Z}_7$.
2. What is the order of the group? (Hint: Do not miss the identity element $O$)
3. Given the element $\alpha = (0,3)$, determine the order of $\alpha$. Is $\alpha$ a primitive element?

## 5 Diffie-Hellman using ECC
Alice and Bob agree to use the elliptic curve $E\ :\ y^2 = x^3 +x+1$ and point $P = (5,4) \in E$. Alice chooses her secret number $a = 5$, whereas Bob chooses $b = 7$.


__Your Task__: Describe the key agreement protocol step by step using the above assumptions about $a$ and $b$. What is the common secret key?

## 6 Elliptic Curve Discrete Logarithm Problem (ECDLP)
Assume Mallory intercepts the message $A = (10,6)$ from Alice to Bob and $B = (7,11)$ from Bob to Alice. He also knows the elliptic curve $E\ :\ y^2 \equiv x^3 +2x+2 \mod 17$, which forms a cyclic group of order 19, and point $P = (5,1)$.

__Your Task__: Suppose Mallory wants to know the common key. Describe his steps to find this key!

In [3]:
(5**3+10+2)%17

1

$P$ is therefore located on $E$.

Mallory now calculates the remainder group, with somethin akin to the square-and-multiply algorithm:
$$ 1P = P $$
$$ 2P = P + P $$
$$ 3P = 2P + P $$
$$ 4P = 2P + 2P $$
$$ 5P = 4P + P $$
$$ \vdots $$

In [4]:
def ecc_add(a, b, point_1, point_2):
    assert isinstance(point_1, tuple)
    assert isinstance(point_2, tuple)
    x1,y1 = point_1
    x2,y2 = point_2
    if (x1 == x2):
        if (y1 == -y2):
            return (0,0)
        if (y1 == y2 and y1 != 0):
            s = (3 * x1 ** 2 + a)/(2 * y1)
    else:
        s = (y2 - y1)/(x2 - x1)
    x3 = s ** 2 - x1 - x2
    y3 = s * (x1 - x3) - y1
    return (x3,y3)

def ecc_double(a, b, point):
    assert isinstance(point, tuple)
    return ecc_add(a, b, point, point)

# Test for y^2 = x^3 + 73 (a,b = 1,73), should result in (-3.5555555555, -5.29629629)
print(ecc_double(1,73,(2,9)))

(-3.478395061728395, -5.04338134430727)


And as we are computing in a remainder group we have to change our program as follows:

In [15]:
import math

# Source: https://stackoverflow.com/questions/4798654/modular-multiplicative-inverse-function-in-python
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    # If a is negative, replace it with an equivalent positive integer in the same remainder class
    if a < 0:
        a = a + m
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

def ecc_add(a, b, p, point_1, point_2):
    assert isinstance(point_1, tuple)
    assert isinstance(point_2, tuple)
    x1,y1 = point_1
    x2,y2 = point_2
    if (x1 == x2):
        if (y1 == -y2):
            return (0,0)
        if (y1 == y2 and y1 != 0):
            s = ((3 * x1 ** 2 + a) * modinv((2 * y1), p)) % p
    else:
        s = ((y2 - y1)* modinv((x2 - x1), p)) % p
    x3 = (s ** 2 - x1 - x2) % p
    y3 = (s * (x1 - x3) - y1) % p
    return (x3,y3)

def ecc_double(a, b, p, point):
    assert isinstance(point, tuple)
    return ecc_add(a, b, p, point, point)

def ecc_multiply(a,b,p, point, n):
    assert isinstance(point, tuple)
    result = point
    if (n == 0):
        result = (0,0)
    # if n = 1, we just return the point
    if (n != 1):
        # if n is bigger than 1 we differentiate between odd and even n
        if (n%2 == 0):
            result = ecc_multiply(a,b,p,ecc_double(a,b,p,point),n/2)
        else:
            result = ecc_add(a,b,p,ecc_multiply(a,b,p,point, n-1), point)
    return result

# Test for y^2 = x^3 + x + 1 mod 5 (a,b = 1,1), and Generator (0,1) should result in (4,2)
print("Test should result in (4,2): " + str(ecc_double(1,1,5,(0,1))))

a,b,p = 2,2,17
P = (5,1)
for i in range(1,19):
    print(str(i) + "*P: " + str(ecc_multiply(a,b,p, P, i)))

Test should result in (4,2): (4, 2)
1*P: (5, 1)
2*P: (6, 3)
3*P: (10, 6)
4*P: (3, 1)
5*P: (9, 16)
6*P: (16, 13)
7*P: (0, 6)
8*P: (13, 7)
9*P: (7, 6)
10*P: (7, 11)
11*P: (13, 10)
12*P: (0, 11)
13*P: (16, 4)
14*P: (9, 1)
15*P: (3, 16)
16*P: (10, 11)
17*P: (6, 14)
18*P: (5, 16)


As Alice's key $Q_A = n_A\cdot P$ and Bob's $Q_B = n_B \cdot P$, Mallory is able to looks up the corresponding values of $n$ for the points $Q_A = (10,6)$ and $Q_B = (7,11)$. Which are:
$$n_A = 3$$
$$n_B = 10$$

She is thus able to compute the common key $n_A\cdot Q_B = n_B\cdot Q_A = $

In [14]:
print("nA*QB: " + str(ecc_multiply(2,2,17,(7,11),3)))
print("nB*QA: " + str(ecc_multiply(2,2,17,(10,6),10)))

nA*QB: (13, 10)
nB*QA: (13, 10)
